In [1]:
##---------------------------------------------------------------------------------
## Summary : Implementing the Conv neural nets for sentence classification
## Author  : Srinivas Venkata Vemparala
## Source  : https://github.com/neubig/nn4nlp-code
##---------------------------------------------------------------------------------

%matplotlib inline
import numpy as np
import pandas as pd
import dynet as dy
import time 
import matplotlib.pyplot as plt
import random
from collections import defaultdict


np.set_printoptions(linewidth=np.nan, threshold=np.nan)

In [2]:
# lets convert the words into integer
# The default dictionary takes a function as input and outupts 
# it if key is not present in the map.
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))

# create an unknown token. As this is the first it will be 0
UNK =w2i['<unk>']
print(UNK)

# Lets write a method to read the data.
# This method returns the list of [features, label].
# Here the features are integer ids of words and tags are labels
def readDataSet(fileName):
    retList = []
    with open(fileName,'r+') as f:
        for line in f:
            tag,words = line.lower().strip().split(' ||| ')
            words = words.split(" ")
            # now get the features which is the integerIds of words
            features = [w2i[x] for x in words]
            label = t2i[tag]
            
            # add the data to the list
            retList.append([words,features,label])
    return retList

0


In [3]:
# Read the data
train = readDataSet('../data/classes/train.txt')
test = readDataSet("../data/classes/test.txt")
nWords = len(w2i)
nTags = len(t2i)

# lets freeze the dictionary
w2i = defaultdict(lambda:UNK, len(w2i))

print(nWords,':',nTags)

18648 : 5


In [4]:
# lets define an embedding size and a filter size
nEmb = 10            # length of the embedding vector for each word
nFilter = 8         # number of filters
winSize = 3          # this is similar to N in N-grams.


# lets declare the model and the trainer.
model = dy.Model()
trainer = dy.AdamTrainer(model)


# lets declare the parameters which will be added to the model
W_emb = model.add_lookup_parameters((nWords,1,1,nEmb))   # the features are on z-axis :)

W_cnn = model.add_parameters((1,winSize,nEmb,nFilter))   # the weights of CNN layer
b_cnn = model.add_parameters((nFilter))                  # biases of the CNN layer

W_sm = model.add_parameters((nTags,nFilter))             # weights of softmax layer
b_sm = model.add_parameters((nTags))

In [5]:
# lets write a method to compute the scores given a sentence
def computeScores(sent):
    # renew the computation graph
    dy.renew_cg()
    
    # get the parameters to computation graph
    weightsCNN = dy.parameter(W_cnn)
    biasesCNN = dy.parameter(b_cnn)
    weightsSM = dy.parameter(W_sm)
    biasesSM = dy.parameter(b_sm)
    
    # if the sent is smaller than the window size then we will have problems
    # if that is case add enough UNK
    if len(sent) < winSize:
        sent += [0] * (winSize-len(sent))

    embed = [W_emb[x] for x in sent]
    cnnInput = dy.concatenate(embed,d=1) # concatenate the embeddings in column dimension
    cnnOutput = dy.conv2d_bias(cnnInput,weightsCNN,biasesCNN,is_valid=False,stride=(1,1))
    
    # do max pooling
    poolOut = dy.max_dim(cnnOutput,d=1)               # get the output of pooling
    poolOut = dy.reshape(poolOut, (nFilter,))   # reshape it to get the filter size
    
    # apply relu function
    poolOut = dy.rectify(poolOut)
    
    return (weightsSM*poolOut+biasesSM)

In [10]:
def calPredictAndActivations(words,wordIds,tag):
     #renew the computation graph
    dy.renew_cg()
    
    # bring the parameters to computation graph
    weightsCNN = dy.parameter(W_cnn)
    biasesCNN = dy.parameter(b_cnn)
    weightsSM = dy.parameter(W_sm)
    biasesSM = dy.parameter(b_sm)
    
    # if the sent is smaller than the window size then we will have problems
    # if that is case add enough UNK
    if len(wordIds) < winSize:
        wordIds += [0] * (winSize-len(wordIds))
    
     #lets get the embeddings for words
    emb = [W_emb[x] for x in wordIds]
    
    cnnInput = dy.concatenate(emb,d=1) # concatenate the embeddings in column dimension
    cnnOutput = dy.conv2d_bias(cnnInput,weightsCNN,biasesCNN,is_valid=False,stride=(1,1))
    
    # reshape the cnnOutput to len(wIds) and nFilters
    filters = (dy.reshape(cnnOutput,(len(wordIds),nFilter))).npvalue()
    activations = np.argmax(filters,axis=0)
    
    # do max pooling
    poolOut = dy.max_dim(cnnOutput,d=1)               # get the output of pooling
    poolOut = dy.reshape(poolOut, (nFilter,))   # reshape it to get the filter size
    poolOut = dy.rectify(poolOut)               # apply relu function
    
    scores = (weightsSM * poolOut + biasesSM).npvalue()
    
    print('%d ||| %s' % (tag, ' '.join(words)))
    predict = np.argmax(scores)
    print(display_activations(words, activations))
    print('scores=%s, predict: %d' % (scores, predict))
    features = poolOut.npvalue()
    W = weightsSM.npvalue()
    bias = biasesSM.npvalue()

    print('  bias=%s' % bias)
    contributions = W * features
    print(' very bad (%.4f): %s' % (scores[0], contributions[0]))
    print('      bad (%.4f): %s' % (scores[1], contributions[1]))
    print('  neutral (%.4f): %s' % (scores[2], contributions[2]))
    print('     good (%.4f): %s' % (scores[3], contributions[3]))
    print('very good (%.4f): %s' % (scores[4], contributions[4]))

In [14]:

def display_activations(words, activations):
    pad_begin = (winSize - 1) / 2
    pad_end = winSize - 1 - pad_begin
    words_padded = ['pad' for i in range(int(pad_begin))] + words + ['pad' for i in range(int(pad_end))]

    ngrams = []
    for act in activations:
        ngrams.append('[' + ', '.join(words_padded[act:act+winSize]) + ']')

    return ngrams

In [8]:
# Now lets perform the the training
bestTestAccuracy = 0
lastTestAccuracy = 0

for i in range(25):
    # Perform the shuffling of the training data
    random.shuffle(train)
    
    # initialize the training loss and time
    startTime = time.time()
    trainLoss = 0
    trainCorrect = 0
    
    # train
    for words,wordIds,tag in train:
        scores = computeScores(wordIds)
        predict = np.argmax(scores.npvalue())
        if(predict==tag):
            trainCorrect = trainCorrect+1
        
        loss = dy.pickneglogsoftmax(scores,tag)
        trainLoss += loss.value()
        
        loss.backward()
        trainer.update()
    print('At iteration ',i,' the training accuracy is : ',(trainCorrect/len(train)),' time taken : ',(time.time()-startTime))
    
    testLoss = 0
    testCorrect = 0
    # test
    for words,wordIds,tag in test:
        scores = computeScores(wordIds)
        predict = np.argmax(scores.npvalue())
        if(predict==tag):
            testCorrect += 1
    testAccuracy = testCorrect/len(test)
    if(testAccuracy>bestTestAccuracy):
        bestTestAccuracy = testAccuracy
    
    if(testAccuracy<lastTestAccuracy):
        trainer.learning_rate = trainer.learning_rate/2
    lastTestAccuracy = testAccuracy
    
    print('At iteration ',i,' the test accuracy is : ',(testAccuracy),' time taken : ',(time.time()-startTime))
    

At iteration  0  the training accuracy is :  0.25152153558052437  time taken :  3.7631001472473145
At iteration  0  the test accuracy is :  0.2493212669683258  time taken :  4.195470094680786
At iteration  1  the training accuracy is :  0.293188202247191  time taken :  3.9597666263580322
At iteration  1  the test accuracy is :  0.25746606334841626  time taken :  4.439058303833008
At iteration  2  the training accuracy is :  0.32092696629213485  time taken :  4.3743016719818115
At iteration  2  the test accuracy is :  0.26334841628959277  time taken :  4.993359088897705
At iteration  3  the training accuracy is :  0.34808052434456926  time taken :  5.941772222518921
At iteration  3  the test accuracy is :  0.2823529411764706  time taken :  6.488176584243774
At iteration  4  the training accuracy is :  0.38085205992509363  time taken :  5.152003765106201
At iteration  4  the test accuracy is :  0.2846153846153846  time taken :  5.834702730178833
At iteration  5  the training accuracy is 

In [16]:
for words, wids, tag in test:
    calPredictAndActivations(words, wids, tag)

2 ||| effective but too-tepid biopic
['[effective, but, too-tepid]', '[too-tepid, biopic, pad]', '[but, too-tepid, biopic]', '[effective, but, too-tepid]', '[effective, but, too-tepid]', '[effective, but, too-tepid]', '[pad, effective, but]', '[but, too-tepid, biopic]']
scores=[ 0.76864535 -0.52596223 -0.47078153 -0.67197961 -1.82972145], predict: 0
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (0.7686): [-1.10063789 -1.42231119  1.76874035  0.90789443 -0.17503114  0.28777322  0.27072805 -0.30150848]
      bad (-0.5260): [ 1.94522736 -0.13375952  2.11761106 -1.42362651 -0.20992215 -0.14851171 -2.42537256  0.39761807]
  neutral (-0.4708): [-0.25913688 -0.98140338  0.64908473 -0.63222974  0.06390983  0.02776703  1.25965996 -0.69197179]
     good (-0.6720): [ 1.65936677 -0.34048932 -2.17352545  1.00613136 -0.04358048  0.33097184 -1.20402344  0.48323239]
very good (-1.8297): [ 0.77433207  1.35276029 -0.65109059  0.03090372  0.12538023 -0.04694862 -2.5386729

 very bad (2.4413): [-0.19236557 -1.17040341  2.34838703  0.01881619 -0.27609118  1.45415238  0.18174874 -0.45589623]
      bad (0.2582): [ 0.33997991 -0.11006916  2.81158869 -0.02950478 -0.33112768 -0.75044735 -1.62823322  0.60121883]
  neutral (0.1298): [-0.04529102 -0.80758548  0.86180098 -0.01310302  0.10081029  0.14031009  0.8456516  -1.04629669]
     good (-1.7191): [ 0.29001822 -0.28018472 -2.88582718  0.02085216 -0.06874312  1.67244013 -0.80830096  0.73067205]
very good (-2.5702): [ 1.35335003e-01  1.11317078e+00 -8.64464194e-01  6.40482323e-04  1.97772679e-01 -2.37236969e-01 -1.70429554e+00 -9.88427462e-01]
1 ||| with a cast that includes some of the top actors working in independent film , lovely & amazing involves us because it is so incisive , so bleakly amusing about how we go about our lives .
['[the, top, actors]', '[us, because, it]', '[actors, working, in]', '[lives, ., pad]', '[lives, ., pad]', '[how, we, go]', '[we, go, about]', '[cast, that, includes]']
scores=[ 2.8

  neutral (0.0031): [-0.41914798 -1.64545236  1.02657669 -0.58390261  0.70278502  0.20842611  1.61489584 -0.99458845]
     good (0.3708): [ 2.68398784 -0.57087531 -3.43759523  0.92922349 -0.47923319  2.48435583 -1.5435693   0.69456206]
very good (-0.8712): [ 1.25246444  2.26808125 -1.02974912  0.02854147  1.37874494 -0.35240786 -3.25460244 -0.93957913]
2 ||| the appearance of treebeard and gollum 's expanded role will either have you loving what you 're seeing , or rolling your eyes .
['[will, either, have]', '[will, either, have]', '[you, loving, what]', '[role, will, either]', '[expanded, role, will]', "[you, 're, seeing]", '[either, have, you]', '[you, loving, what]']
scores=[-0.15857768 -2.82506704 -0.96203196 -1.13692069 -2.01480126], predict: 0
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (-0.1586): [-1.22242125 -2.4627556   3.33854475  1.49105554 -2.48495712  1.15239864  0.37390016 -0.8773409 ]
      bad (-2.8251): [ 2.16046285 -0.23160685  3.99

0 ||| britney has been delivered to the big screen safe and sound , the way we like our 20-year-old superstar girls to travel on the fame freeway .
['[superstar, girls, to]', '[been, delivered, to]', '[travel, on, the]', '[travel, on, the]', '[the, big, screen]', '[been, delivered, to]', '[girls, to, travel]', '[freeway, ., pad]']
scores=[-0.83357584  3.55161667 -2.23455143  1.89797318  0.7473582 ], predict: 1
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (-0.8336): [-2.7833125  -3.26128897  2.89170219  0.92191689 -0.32511078  1.45398553  0.21472947 -0.47919612]
      bad (3.5516): [ 4.91912525 -0.30670395  3.46206868 -1.44561446 -0.38991893 -0.75036124 -1.9236979   0.63194585]
  neutral (-2.2346): [-0.65530991 -2.25030924  1.06118444 -0.64199455  0.11870901  0.14029399  0.99910639 -1.09977069]
     good (1.8980): [ 4.19623595 -0.78072512 -3.55348275  1.02167109 -0.08094836  1.67224823 -0.95497797  0.76801515]
very good (0.7474): [ 1.9581446   3.1018122

      bad (-2.5274): [ 3.1525743  -0.22660694  2.46865293 -1.26929394 -1.60512418 -0.7590785  -4.2764792   0.63315485]
  neutral (-0.0463): [-0.41997572 -1.66263166  0.75668519 -0.56369095  0.48867262  0.14192384  2.2210648  -1.1018747 ]
     good (0.0906): [ 2.68928823 -0.57683552 -2.53383639  0.89705862 -0.3332287   1.69167543 -2.12296506  0.76948447]
very good (-2.2059): [ 1.25493782  2.2917611  -0.75902356  0.02755351  0.95869275 -0.23996551 -4.47625335 -1.04093153]
1 ||| morton is , as usual , brilliant .
['[pad, morton, is]', '[brilliant, ., pad]', '[morton, is, ,]', '[pad, morton, is]', '[morton, is, ,]', '[as, usual, ,]', '[brilliant, ., pad]', '[pad, morton, is]']
scores=[ 0.74531639  2.53454542 -0.85795271 -0.56276417 -0.07371995], predict: 1
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (0.7453): [-0.80441875 -1.4915477   1.90453733  0.         -0.          0.86608937  0.03676113 -0.29910326]
      bad (2.5345): [ 1.42170043 -0.14027079  2.28

...

      bad (1.1795): [ 2.86967763 -0.08564834  1.85890183 -0.         -1.75289638 -0.06864264 -1.87384778  0.87715689]
  neutral (-0.3542): [-0.38228915 -0.62840809  0.56978584 -0.          0.53366119  0.01283402  0.97321585 -1.52650965]
     good (-0.1432): [ 2.44796459 -0.21802069 -1.90798514  0.         -0.36390666  0.15297635 -0.93023097  1.06602453]
very good (-1.1639): [ 1.14232581  0.86619378 -0.57154664  0.          1.04695267 -0.02169982 -1.9613839  -1.4420805 ]
4 ||| unfortunately , heartbreak hospital wants to convey the same kind of haughtiness in its own sketchy material but this territory has already been explored previously with better aplomb and sardonic wit .
['[kind, of, haughtiness]', '[better, aplomb, and]', '[territory, has, already]', '[the, same, kind]', '[sketchy, material, but]', '[territory, has, already]', '[but, this, territory]', '[but, this, territory]']
scores=[-1.31310499 -2.14040375 -2.02280807  0.39125502 -0.54406428], predict: 3
  bias=[ 0.5329982  -0.

     good (2.6234): [ 5.01870093 -0.26018563 -2.15926224  0.49156923 -0.32540736  1.57792497 -1.63086628  0.30103617]
very good (-0.6123): [ 2.34194222  1.03371462 -0.64681797  0.01509874  0.93619091 -0.22382992 -3.43866735 -0.40723115]
0 ||| with danilo donati 's witty designs and dante spinotti 's luscious cinematography , this might have made a decent children 's movie -- if only benigni had n't insisted on casting himself in the title role .
['[might, have, made]', '[made, a, decent]', "[n't, insisted, on]", '[role, ., pad]', '[title, role, .]', '[made, a, decent]', '[this, might, have]', '[witty, designs, and]']
scores=[-0.24465132 -1.62287784 -0.99241817  1.01678824 -1.92810798], predict: 3
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (-0.2447): [-1.75427453 -2.3915888   2.6321289   1.00851928 -1.79242051  1.95857248  0.37185696 -0.81044367]
      bad (-1.6229): [ 3.10044099 -0.22491405  3.15129651 -1.5814116  -2.14972475 -1.01076445 -3.33135661 

 very bad (-1.4460): [-2.02968991 -2.73719766  2.56070365  1.58181457 -2.19288252  1.42460915  0.2185737  -0.80494777]
      bad (-0.9918): [ 3.58720011 -0.25741642  3.06578317 -2.48036895 -2.63001556 -0.73520091 -1.95813719  1.06153491]
  neutral (-2.3271): [-0.47787516 -1.88868304  0.93971602 -1.10152699  0.80069606  0.13745949  1.01699304 -1.84738136]
     good (2.0314): [ 3.06004366 -0.65526207 -3.14673353  1.7529717  -0.54599929  1.63846206 -0.97207461  1.29010246]
very good (0.4634): [ 1.42794829  2.60334891 -0.94262002  0.05384322  1.57082977 -0.23241715 -2.04961085 -1.74520524]
3 ||| a predictable and stereotypical little b-movie .
['[and, stereotypical, little]', '[a, predictable, and]', '[little, b-movie, .]', '[predictable, and, stereotypical]', '[predictable, and, stereotypical]', '[a, predictable, and]', '[stereotypical, little, b-movie]', '[a, predictable, and]']
scores=[-1.83191931  0.91421115 -2.2064569   3.25463343 -0.39993808], predict: 3
  bias=[ 0.5329982  -0.645226

     good (-0.2824): [ 2.47855868 -0.54426858 -3.07404253  0.46681055 -0.39176939  1.44213587 -1.25815683  0.98843777]
very good (-0.8776): [ 1.15660233  2.16237301 -0.92084506  0.01433827  1.12711324 -0.20456813 -2.65281273 -1.33712386]
3 ||| just because it really happened to you , honey , does n't mean that it 's interesting to anyone else .
["[n't, mean, that]", '[just, because, it]', '[really, happened, to]', "[does, n't, mean]", "[does, n't, mean]", "[it, 's, interesting]", "[does, n't, mean]", '[honey, ,, does]']
scores=[-1.00230885 -2.93889952 -0.31344071  2.06145525 -0.86828756], predict: 3
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (-1.0023): [-1.61753328 -1.2528064   0.8080858   0.9099925  -1.50367055  1.20195676  0.2956473  -0.37697915]
      bad (-2.9389): [ 2.85876946 -0.11781865  0.96747464 -1.4269164  -1.80341486 -0.62029625 -2.64861678  0.49714595]
  neutral (-0.3134): [-0.380836   -0.86444404  0.29654785 -0.63369077  0.5490413   0.1

['[woman, sporting, them]', '[banger, sisters, inevitably]', '[like, those, to]', '[woman, sporting, them]', '[woman, sporting, them]', '[pad, like, those]', '[sporting, them, .]', '[the, woman, sporting]']
scores=[-1.64398706 -3.82056284  0.93212271  1.53426516 -1.77530718], predict: 3
  bias=[ 0.5329982  -0.6452266   0.09353867 -0.39006341 -0.22268578]
 very bad (-1.6440): [-2.5300906  -1.65039262  1.80060128  0.97389048 -2.57257916  1.69375828  0.52600668 -0.41817948]
      bad (-3.8206): [ 4.47159007 -0.15520916  2.15575633 -1.5271118  -3.08540159 -0.87410125 -4.71233836  0.5514794 ]
  neutral (0.9321): [-0.59569073 -1.13878095  0.66077692 -0.67818736  0.93933623  0.16342949  2.44743593 -0.95973553]
     good (1.5343): [ 3.81446823 -0.3950901  -2.21267792  1.07926837 -0.64053882  1.94801409 -2.33933786  0.67022283]
very good (-1.7753): [ 1.77999532  1.56968855 -0.66281891  0.03315016  1.84281825 -0.27632735 -4.93247352 -0.90665388]
2 ||| we may get the full visceral impact of a rut